In [2]:
import os
import sys
import pandas as pd
import numpy as np
import json
import copy

In [3]:
ALL_KEYS = ["fold", "uid", "questions", "concepts", "responses", "timestamps",
            "usetimes", "selectmasks", "is_repeat", "qidxs", "rest", "orirow", "cidxs"]
ONE_KEYS = ["fold", "uid"]

In [18]:
df = pd.read_csv("/mnt/cfs/zhanbojun/pykt-toolkit/data/ednet/train_valid.csv")


In [19]:
# 查看df中uid这一列有多少个不同的值 939
df['uid'].nunique()

3760

In [20]:
df2 = pd.DataFrame(columns=df.columns)
seen_concepts = set()
rows_to_drop = []

In [7]:
def check_new_concepts(row):
    global df2
    concepts = row['concepts'].split(',')
    for concept in concepts:
        if concept not in seen_concepts:
            df2 = pd.concat([df2, pd.DataFrame(row).T])
            seen_concepts.add(concept)
            rows_to_drop.append(row.name)
            break


In [21]:
df.apply(check_new_concepts, axis=1)
df2

,fold,uid,questions,concepts,responses,timestamps,usetimes,is_repeat
0,0,1762,"125,125,125,126,126,126,127,127,127,128,128,12...","52,50,4,90,54,3,91,54,3,1,92,4,21,13,79,3,21,7...","1,1,1,0,0,0,1,1,1,0,0,0,0,1,1,1,1,0,0,0,0,0,0,...","1511519233047,1511519233047,1511519233047,1511...","19000,19000,19000,28000,28000,28000,22000,2200...","0,1,1,0,1,1,0,1,1,0,1,1,1,0,1,1,1,0,1,1,0,1,1,..."
1,0,2522,"160,830,1291,1291,1291,1291,5450,6184,5053,505...","100,150,141,54,13,4,10,11,132,50,3,124,125,19,16","0,0,0,0,0,0,1,0,1,1,1,1,1,1,1","1556728376496,1556728400052,1556728426063,1556...","31000,20000,22000,22000,22000,22000,20000,2100...","0,0,0,1,1,1,0,0,0,1,1,0,1,1,1"
2,0,1313,"838,10641,10419,2097,1702,5057","150,9,129,26,10,26","0,0,0,0,0,1","1540680030887,1540680081552,1540680098539,1540...","53000,46000,13000,16000,9000,7000","0,0,0,0,0,0"
3,0,1985,"125,125,125,126,126,126,127,127,127,128,128,12...","52,50,4,90,54,3,91,54,3,1,92,4,21,79,3,21,83,3...","1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,1,1,1,1,...","1531236749783,1531236749783,1531236749783,1531...","20000,20000,20000,22000,22000,22000,27000,2700...","0,1,1,0,1,1,0,1,1,0,1,1,1,0,1,1,0,1,1,0,1,1,1,..."
4,0,146,"125,125,125,126,126,126,127,127,127,128,128,12...","52,50,4,90,54,3,91,54,3,1,92,4,21","1,1,1,0,0,0,0,0,0,1,1,1,1","1534549225006,1534549225006,1534549225006,1534...","15000,15000,15000,27000,27000,27000,50000,5000...","0,1,1,0,1,1,0,1,1,0,1,1,1"
...,...,...,...,...,...,...,...,...
744,0,1155,"2144,5776,7564,7183,10923,5091,1700,10669,6847...","109,100,9,38,40,7,6,11,6,38,26,179,89,9,6,86,2...","0,1,1,0,0,0,1,0,0,1,0,1,0,1,1,0,0,1,1,1,1,1,1,...","1540214009128,1540300199177,1540300213980,1540...","42000,14000,12000,36000,23000,40000,10000,5400...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,..."
895,1,2312,"125,125,125,126,126,126,127,127,127,128,128,12...","52,50,4,90,54,3,91,54,3,1,92,4,21,79,3,21,83,3...","1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","1512440036407,1512440036407,1512440036407,1512...","24000,24000,24000,23000,23000,23000,23000,2300...","0,1,1,0,1,1,0,1,1,0,1,1,1,0,1,1,0,1,1,0,1,1,1,..."
929,1,1333,"317,317,317,318,318,318,319,319,319,320,320,32...","132,50,3,133,50,4,132,50,3,134,50,13,19,135,50...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...","1502583339875,1502583339875,1502583339875,1502...","9000,9000,9000,12000,12000,12000,5000,5000,500...","0,1,1,0,1,1,0,1,1,0,1,1,1,0,1,1,0,1,1,0,1,1,1,..."
994,1,4004,"11495,11476,8677,426,2903,2903,2903,289,10516,...","26,129,11,38,91,54,19,26,26,120,40,75,48,47,75...","1,1,0,1,1,1,1,1,0,0,0,1,1,1,1,1,1,0,0,0,1,1,1,...","1566126164175,1566126176835,1566126200119,1566...","23000,7000,19000,17000,24000,24000,24000,12000...","0,0,0,0,0,1,1,0,0,0,0,0,1,1,0,1,1,0,1,1,0,1,1,..."


In [22]:
df.apply(check_new_concepts, axis=1)
df = df.drop(rows_to_drop)
total_students = df2.shape[0] * 4
remain_students = total_students - df2.shape[0]
# 从df中随机抽取remain_students行添加到df2中
random_rows = df.sample(n=remain_students)
df2 = pd.concat([df2,random_rows])
df2.reset_index(drop=True)

,fold,uid,questions,concepts,responses,timestamps,usetimes,is_repeat
0,0,1762,"125,125,125,126,126,126,127,127,127,128,128,12...","52,50,4,90,54,3,91,54,3,1,92,4,21,13,79,3,21,7...","1,1,1,0,0,0,1,1,1,0,0,0,0,1,1,1,1,0,0,0,0,0,0,...","1511519233047,1511519233047,1511519233047,1511...","19000,19000,19000,28000,28000,28000,22000,2200...","0,1,1,0,1,1,0,1,1,0,1,1,1,0,1,1,1,0,1,1,0,1,1,..."
1,0,2522,"160,830,1291,1291,1291,1291,5450,6184,5053,505...","100,150,141,54,13,4,10,11,132,50,3,124,125,19,16","0,0,0,0,0,0,1,0,1,1,1,1,1,1,1","1556728376496,1556728400052,1556728426063,1556...","31000,20000,22000,22000,22000,22000,20000,2100...","0,0,0,1,1,1,0,0,0,1,1,0,1,1,1"
2,0,1313,"838,10641,10419,2097,1702,5057","150,9,129,26,10,26","0,0,0,0,0,1","1540680030887,1540680081552,1540680098539,1540...","53000,46000,13000,16000,9000,7000","0,0,0,0,0,0"
3,0,1985,"125,125,125,126,126,126,127,127,127,128,128,12...","52,50,4,90,54,3,91,54,3,1,92,4,21,79,3,21,83,3...","1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,1,1,1,1,...","1531236749783,1531236749783,1531236749783,1531...","20000,20000,20000,22000,22000,22000,27000,2700...","0,1,1,0,1,1,0,1,1,0,1,1,1,0,1,1,0,1,1,0,1,1,1,..."
4,0,146,"125,125,125,126,126,126,127,127,127,128,128,12...","52,50,4,90,54,3,91,54,3,1,92,4,21","1,1,1,0,0,0,0,0,0,1,1,1,1","1534549225006,1534549225006,1534549225006,1534...","15000,15000,15000,27000,27000,27000,50000,5000...","0,1,1,0,1,1,0,1,1,0,1,1,1"
...,...,...,...,...,...,...,...,...
747,1,1508,"1907,7200,4,423,5053,5053,5053,432,7721","34,26,7,26,132,50,3,10,9","1,1,0,0,1,1,1,0,1","1546408381847,1546408410704,1546408440091,1546...","18000,25000,27000,24000,23000,23000,23000,1800...","0,0,0,0,0,1,1,0,0"
748,1,921,"10223,4296,4296,4296,4296,10202,1702,4712,4712...","129,14,15,4,21,26,10,60,110,3,16,12,1,62,16,3","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0","1553096147539,1553096189016,1553096189016,1553...","52000,37000,37000,37000,37000,40000,43000,3200...","0,0,1,1,1,0,0,0,1,1,1,0,1,1,1,1"
749,4,266,"4231,4599,1612,5117,5117,5117,5117,6172,6172,6...","150,11,6,1,18,16,3,1,128,21,19,12,13,124,125,2...","0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,1,1,1,1,1","1541274651315,1541274669510,1541274696033,1541...","18000,15000,24000,16000,16000,16000,16000,1600...","0,0,0,0,1,1,1,0,1,1,1,0,1,1,1,1,1,0,1,1,1,1"
750,3,3090,"7276,2492,1633,5091,3247,7110,5249,2369,3011,182","10,120,26,7,26,9,44,29,120,35","0,1,1,0,0,0,0,0,0,0","1547635872005,1547635903660,1547635925973,1547...","28000,29000,15000,12000,17000,18000,17000,1300...","0,0,0,0,0,0,0,0,0,0"


In [10]:
def KFold_split(df, k=5):
    df = df.sample(frac=1.0, random_state=1024)
    datanum = df.shape[0]
    test_ratio = 1 / k
    test_num = int(datanum * test_ratio)
    rest = datanum % k

    start = 0
    folds = []
    for i in range(0, k):
        if rest > 0:
            end = start + test_num + 1
            rest -= 1
        else:
            end = start + test_num
        folds.extend([i] * (end - start))
        print(f"fold: {i+1}, start: {start}, end: {end}, total num: {datanum}")
        start = end
    # report
    finaldf = copy.deepcopy(df)
    finaldf["fold"] = folds
    return finaldf

def calStatistics(df, stares, key):
    allin, allselect = 0, 0
    allqs, allcs = set(), set()
    for i, row in df.iterrows():
        rs = row["responses"].split(",")
        curlen = len(rs) - rs.count("-1")
        allin += curlen
        if "selectmasks" in row:
            ss = row["selectmasks"].split(",")
            slen = ss.count("1")
            allselect += slen
        if "concepts" in row:
            cs = row["concepts"].split(",")
            fc = list()
            for c in cs:
                cc = c.split("_")
                fc.extend(cc)
            curcs = set(fc) - {"-1"}
            allcs |= curcs
        if "questions" in row:
            qs = row["questions"].split(",")
            curqs = set(qs) - {"-1"}
            allqs |= curqs
    stares.append(",".join([str(s)
                  for s in [key, allin, df.shape[0], allselect]]))
    return allin, allselect, len(allqs), len(allcs), df.shape[0]


In [23]:
df2 = KFold_split(df2, k=5)
stares = []

fold: 1, start: 0, end: 151, total num: 752
fold: 2, start: 151, end: 302, total num: 752
fold: 3, start: 302, end: 452, total num: 752
fold: 4, start: 452, end: 602, total num: 752
fold: 5, start: 602, end: 752, total num: 752


In [24]:
ins, ss, qs, cs, seqnum = calStatistics(
        df2, stares, "original train+valid")
print(
        f"train+valid original interactions num: {ins}, select num: {ss}, qs: {qs}, cs: {cs}, seqnum: {seqnum}")

train+valid original interactions num: 332355, select num: 0, qs: 11351, cs: 188, seqnum: 752


In [13]:
def save_dcur(row, effective_keys):
    dcur = dict()
    for key in effective_keys:
        if key not in ONE_KEYS:
            dcur[key] = row[key].split(",")#[int(i) for i in row[key].split(",")]
        else:
            dcur[key] = row[key]
    return dcur

In [14]:
def generate_sequences(df, effective_keys, min_seq_len=3, maxlen=200, pad_val=-1):
    save_keys = list(effective_keys) + ["selectmasks"]
    dres = {"selectmasks": []}
    dropnum = 0
    for i, row in df.iterrows():
        dcur = save_dcur(row, effective_keys)

        rest, lenrs = len(dcur["responses"]), len(dcur["responses"])
        j = 0
        while lenrs >= j + maxlen:
            rest = rest - (maxlen)
            for key in effective_keys:
                dres.setdefault(key, [])
                if key not in ONE_KEYS:
                    # [str(k) for k in dcur[key][j: j + maxlen]]))
                    dres[key].append(",".join(dcur[key][j: j + maxlen]))
                else:
                    dres[key].append(dcur[key])
            dres["selectmasks"].append(",".join(["1"] * maxlen))

            j += maxlen
        if rest < min_seq_len:  # delete sequence len less than min_seq_len
            dropnum += rest
            continue

        pad_dim = maxlen - rest
        for key in effective_keys:
            dres.setdefault(key, [])
            if key not in ONE_KEYS:
                paded_info = np.concatenate(
                    [dcur[key][j:], np.array([pad_val] * pad_dim)])
                dres[key].append(",".join([str(k) for k in paded_info]))
            else:
                dres[key].append(dcur[key])
        dres["selectmasks"].append(
            ",".join(["1"] * rest + [str(pad_val)] * pad_dim))

    # after preprocess data, report
    dfinal = dict()
    for key in ALL_KEYS:
        if key in save_keys:
            dfinal[key] = dres[key]
    finaldf = pd.DataFrame(dfinal)
    print(f"dropnum: {dropnum}")
    return finaldf

In [25]:
effective_keys = df2.keys()
split_seqs = generate_sequences(
    df2, effective_keys=effective_keys)
ins, ss, qs, cs, seqnum = calStatistics(
    split_seqs, stares, "train+valid sequences")
print(
    f"train+valid sequences interactions num: {ins}, select num: {ss}, qs: {qs}, cs: {cs}, seqnum: {seqnum}")

dropnum: 2
train+valid sequences interactions num: 332353, select num: 332353, qs: 11351, cs: 188, seqnum: 2209


In [26]:
split_seqs

,fold,uid,questions,concepts,responses,timestamps,usetimes,selectmasks,is_repeat
0,0,2070,"125,125,125,126,126,126,127,127,127,128,128,12...","52,50,4,90,54,3,91,54,3,1,92,4,21,79,3,21,83,3...","1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,1,1,1,...","1516002866888,1516002866888,1516002866888,1516...","13000,13000,13000,26000,26000,26000,30000,3000...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...","0,1,1,0,1,1,0,1,1,0,1,1,1,0,1,1,0,1,1,0,1,1,1,..."
1,0,2070,"8369,8370,8372,8371,4113,4114,4115,4116,1875,1...","38,26,9,39,8,103,39,40,13,158,13,42,13,6,13,39...","1,1,1,1,1,0,1,1,1,1,0,0,0,0,1,1,1,0,0,1,1,1,0,...","1517812331214,1517812331750,1517812332323,1517...","27000,27000,27000,27000,48750,48750,48750,4875...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...","0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,..."
2,0,2070,"437,1306,1306,1306,1306,338,338,338,338,6318,6...","16,133,50,13,3,56,50,13,4,134,50,3,53,54,16,91...","1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...","1518574072469,1518574137939,1518574137939,1518...","23000,23000,23000,23000,23000,27000,27000,2700...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...","1,0,1,1,1,0,1,1,1,0,1,1,0,1,1,0,1,1,1,0,1,1,1,..."
3,0,2070,"101,101,101,101,102,102,102,102,100,100,100,10...","80,13,81,3,80,13,79,3,80,13,58,3,171,81,4,171,...","0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,1,1,...","1519888912010,1519888912010,1519888912010,1519...","23333,23333,23333,23333,23333,23333,23333,2333...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...","0,1,1,1,0,1,1,1,0,1,1,1,0,1,1,0,1,1,0,1,1,0,1,..."
4,0,2070,"9015,9015,9015,9014,9014,9014,3948,3948,3948,3...","85,58,4,85,83,4,84,13,81,3,84,13,59,3,84,13,58...","1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,...","1520310903980,1520310903980,1520310903980,1520...","13000,13000,13000,13000,13000,13000,21000,2100...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...","0,1,1,0,1,1,0,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1,0,..."
...,...,...,...,...,...,...,...,...,...
2204,4,252,"1012,2157,5434,7564,6172,6172,6172,6172,6423,6...","114,40,27,9,1,128,21,19,14,15,3,19,16,3,12,14,...","0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...","1553222402195,1553222421561,1553222444934,1553...","19000,15000,19000,18000,18000,18000,18000,1800...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...","0,0,0,0,0,1,1,1,0,1,1,1,0,1,1,1,1,0,1,1,1,1,0,..."
2205,4,3909,"838,6437,6437,6437,6437,910,433,433,433,431,43...","150,122,123,19,21,176,133,50,21,63,64,16,19,33...","0,0,0,0,0,1,0,0,0,1,1,1,1,1,0,1,-1,-1,-1,-1,-1...","1566829014583,1566829037209,1566829037209,1566...","15000,18000,18000,18000,18000,23000,22000,2200...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,-1,-1,-1,-1...","0,0,1,1,1,0,0,1,1,0,1,1,1,0,0,0,-1,-1,-1,-1,-1..."
2206,4,5,"181,182,183,184,185,186,187,-1,-1,-1,-1,-1,-1,...","9,35,107,26,7,10,9,-1,-1,-1,-1,-1,-1,-1,-1,-1,...","1,1,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1...","1566744672200,1566744688296,1566744721108,1566...","25000,13000,30000,24000,18000,20000,17000,-1,-...","1,1,1,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1...","0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1..."
2207,4,3148,"125,125,125,126,126,126,127,127,127,128,128,12...","52,50,4,90,54,3,91,54,3,1,92,4,21,79,3,21,83,3...","1,1,1,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,1,1,1,1,...","1532821747075,1532821747075,1532821747075,1532...","30000,30000,30000,26000,26000,26000,25000,2500...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...","0,1,1,0,1,1,0,1,1,0,1,1,1,0,1,1,0,1,1,0,1,1,1,..."


In [27]:
split_seqs.to_csv("/mnt/cfs/zhanbojun/pykt-toolkit/data/ednet/train_valid_sequences_fixed_4.csv", index=False)